# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [8]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [3]:
import os

filename = 'classificado.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo classificado.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [4]:
train = pd.read_excel(filename)
# Colocando as classificações : 0: "Negativo", 1: "Neutro", 2: "Positivo"
train["Classificação"] = train["Classificação"].astype("category")
train["Classificação"].cat.categories = ["Negativo","Neutro","Positivo"]

train.head(5)

,Treinamento,Classificação
0,@anyretro acho que eu amo o de flocos e nutella,Positivo
1,pedi pastel de camarão e sla agr to arrependid...,Positivo
2,algm manda um pote de nutella pra mim que eu f...,Neutro
3,@corinthians_ff torcida modinha time vai mal r...,Neutro
4,"@marciom61326104 cai fora paga pau de chines, ...",Neutro


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
# Colocando as classificações : 0: "Negativo", 1: "Neutro", 2: "Positivo"
test["Classificação"] = test["Classificação"].astype("category")
test["Classificação"].cat.categories = ["Negativo","Neutro","Positivo"]
test.head(5)

,Teste,Classificação
0,@duvirmond tênis? nutella,Negativo
1,como assim tem gente q n come nutella no açaí?...,Positivo
2,"das delícias da abuelita, bolo ferrero! a comb...",Positivo
3,daqui a pouco enfiam nutella e granulado como ...,Neutro
4,biel querendo expulsão da raissa por causa de ...,Negativo


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
#Nesta célula vamos preparar os dados para fazer as probabilidades.



#Twitters do treinamento com suas devidas classificações(Série)
train_negativo = train.loc[train["Classificação"] == "Negativo",:]
train_positivo = train.loc[train["Classificação"] == "Positivo",:]
train_neutro = train.loc[train["Classificação"] == "Neutro",:]

#Twitters do treinamento com suas devidas classificações(Lista)
twitters_negativo = []
twitters_positivo =[]
twitters_neutro = []
twitters_tudo = []

#Palavras do treinamento com suas devidas classificações(Lista)
palavras_negativo = []
palavras_positivo = []
palavras_neutro = []
palavras_tudo = []

# loop para encontrar os twitters
for i in train_positivo.Treinamento:
    twitters_positivo.append(i)
    
for i in train_negativo.Treinamento:
    twitters_negativo.append(i)

for i in train_neutro.Treinamento:
    twitters_neutro.append(i)
    
for i in train.Treinamento:
    twitters_tudo.append(i)

# loop para encontrar as palavras

for i in twitters_positivo:
    word = i.split()
    for t in word:
         palavras_positivo.append(t)
    
for i in twitters_negativo:
    word = i.split()
    for t in word:
         palavras_negativo.append(t)

for i in twitters_neutro:
    word = i.split()
    for t in word:
         palavras_neutro.append(t)
            
for i in twitters_tudo:
    word = i.split()
    for t in word:
         palavras_tudo.append(t)

#Função de limpeza de caracteres irrelevantes para o Nayve-Bayes:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;@/•]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

# Transformando as palavras em texto:
texto_positivo = ""
texto_negativo = ""
texto_neutro = ""
texto_tudo = ""

# o loop para tranformar palavras em texto:
for i in palavras_positivo:
    texto_positivo += " "
    texto_positivo += i

for i in palavras_negativo:
    texto_negativo += " "
    texto_negativo += i

for i in palavras_neutro:
    texto_neutro += " "
    texto_neutro += i
    
for i in palavras_tudo:
    texto_tudo += " "
    texto_tudo += i

#Limpando os textos:
texto_negativo = cleanup(texto_negativo.lower())
texto_neutro = cleanup(texto_neutro.lower())
texto_positivo = cleanup(texto_positivo.lower())
texto_tudo = cleanup(texto_tudo.lower())

#Colocando tudo em uma série e um lista:
serie_tudo = pd.Series(texto_tudo.split())
serie_negativo = pd.Series(texto_negativo.split())
serie_neutro = pd.Series(texto_neutro.split())
serie_positivo = pd.Series(texto_positivo.split())

lista_tudo = list(texto_tudo.split())
lista_negativo = list(texto_negativo.split())
lista_neutro = list(texto_neutro.split())
lista_positivo = list(texto_positivo.split())





In [7]:
# Vamos começar a construir as probabilidades
prob_tudo = serie_tudo.value_counts()
prob_negativo = serie_negativo.value_counts()
prob_neutro = serie_neutro.value_counts()
prob_positivo = serie_positivo.value_counts()

# Quantidade de palavras em cada classificação
T_tudo = len(serie_tudo)
T_negativo = len(serie_negativo)
T_neutro = len(serie_neutro)
T_positivo = len(serie_positivo)

#Uma vez construida as probabilidades podemos começar a pensar na função para a classificação:
#Mas precisamos um alfa e um N para fazer o Laplace smoothing:
#alpha(a):1
#N(N): 10^6

def classifica(frase):
    a = 1
    N = 10**4
    p_negativo = 1
    p_positivo = 1
    p_neutro = 1
    palavras = frase.split()
    for i in palavras:
        
        if i not in lista_negativo:
            p_negativo *= a/(T_negativo+N*a)
        elif i in lista_negativo:
            p_negativo *= a+prob_negativo[i]/(T_negativo+N*a)
            
        if i not in lista_neutro:
            p_neutro *= a/(T_neutro+N*a)
        elif i in lista_neutro:
            p_neutro *= a+prob_neutro[i]/(T_neutro+N*a)
            
        if i not in lista_positivo:
            p_positivo *= a/(T_positivo+N*a)
        elif i in lista_positivo:
            p_positivo *= a+prob_positivo[i]/(T_positivo+N*a)
            
    if p_negativo > p_neutro and p_positivo:
        return "Negativo", p_neutro, p_positivo, p_negativo
    elif p_neutro > p_negativo and p_positivo:
        return "Neutro", p_neutro, p_positivo, p_negativo
    elif p_positivo > p_negativo and p_neutro:
        return "Positivo",  p_neutro, p_positivo, p_negativo

In [9]:
classifica("nuttela com milkshake é bom")

('Negativo',
 4.103847625736903e-09,
 7.465235562391339e-05,
 9.05388205585097e-09)

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**